In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from skimage.transform import estimate_transform

Setting environment for Gordon


In [24]:
stack = 'MD594'

Setting environment for Gordon


In [ ]:
# if os.path.exists('/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/'+stack):
#     os.system('rm -r ' + '/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/'+stack)

In [19]:
transforms = pickle.load(open(os.environ['DATA_DIR'] + '/%s_finalTransfParams.pkl' % stack, 'r'))

In [20]:
new_annotation_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped_midbrainIncluded/'

In [21]:
def transfer_latest_original_to_alignedCropped(stack, sec, username):
    
    res = DataManager.load_annotation(stack=stack, section=sec, username=username, 
                                      annotation_rootdir='/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/')
    if res is None:
        return
    
    annotations, _, timestamp = res
    
    new_cropbox_xmin, new_cropbox_ymin = brainstem_bbox_lookup_midbrain[stack][:2]
    
    T = np.linalg.inv(transforms[sec])
    T[:2, 2] = (T[:2, 2] - [new_cropbox_xmin, new_cropbox_ymin])*32
    
    new_annotations = []
    for pol in annotations:
        new_pol = pol.copy()
        if len(pol['vertices']) == 0:
            continue
        
        vs = pol['vertices']
        new_pol['vertices'] = np.dot(T, np.vstack([vs.T, np.ones((1, vs.shape[0]))]))[:2].T.astype(np.int) 
        
        vs = pol['labelPos']
        new_pol['labelPos'] = np.squeeze(np.dot(T, np.r_[vs, 1]))[:2].T.astype(np.int)
                
        new_annotations.append(new_pol)
        
    DataManager.save_annotation(new_annotations, stack, sec, username=username, timestamp=timestamp, 
                                annotation_rootdir=new_annotation_rootdir)

In [23]:
# transfer labelings on originalLossless to labelings on new alignedCropped

for sec in range(section_number_lookup[stack]):
    transfer_latest_original_to_alignedCropped(stack, sec, username='yuncong')
    transfer_latest_original_to_alignedCropped(stack, sec, username='autoAnnotate')
    transfer_latest_original_to_alignedCropped(stack, sec, username='localAdjusted')
    transfer_latest_original_to_alignedCropped(stack, sec, username='globalAligned')